#### Importación de librerias

In [564]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pycountry
import re
import unicodedata
import requests


#### Cargue de datos

In [565]:
data = pd.read_csv('data/raw_data.csv',delimiter=',')


In [566]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28115 entries, 0 to 28114
Data columns (total 18 columns):
 #   Column                                                                                                                                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                --------------  -----  
 0   Timestamp                                                                                                                                                                                                                             28115 non-null  object 
 1   How old are you?                                                                                             

El conjunto de datos proporcionado tiene 28.115 registros y 18 columnas, de las cuales 16 son de tipo categórico y 2 de tipo numérico (salario anual y compesaciones adicionales). 

#### Limpieza de datos

In [567]:
# Cambiar el nombre de las columnas a español y nombres abreviados
data.columns = ['fecha',
                'edad',
                'industria',
                'titulo_profesional',
                'extra_titulo',
                'salario_anual',
                'compensaciones',
                'moneda',
                'otra_moneda',
                'ingresos_extra',
                'pais',
                'estado',
                'ciudad',
                'años_experiencia_total',
                'años_experiencia_industria',
                'nivel_educacion',
                'género',
                'raza'
                ]



In [568]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28115 entries, 0 to 28114
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   fecha                       28115 non-null  object 
 1   edad                        28115 non-null  object 
 2   industria                   28040 non-null  object 
 3   titulo_profesional          28114 non-null  object 
 4   extra_titulo                7273 non-null   object 
 5   salario_anual               28115 non-null  object 
 6   compensaciones              20796 non-null  float64
 7   moneda                      28115 non-null  object 
 8   otra_moneda                 212 non-null    object 
 9   ingresos_extra              3047 non-null   object 
 10  pais                        28115 non-null  object 
 11  estado                      23077 non-null  object 
 12  ciudad                      28033 non-null  object 
 13  años_experiencia_total      281

In [569]:
# Cambiar el tipo de datos a las columnas
data['salario_anual'] = data['salario_anual'].str.replace(',', '', regex=True).astype(float)
data['compensaciones'] = data['compensaciones'].astype(float)

In [570]:
# Rellenar valores nulos
data['género'] = data['género'].fillna('Prefer not to answer')
data['titulo_profesional'] = data['titulo_profesional'].fillna('NA')
data['titulo_profesional'] = data['titulo_profesional'].fillna('NA')

In [571]:
# Asegurar que las columnas a limpiar sean string
data['pais'] = data['pais'].astype(str)
data['ciudad'] = data['ciudad'].astype(str)
# data['moneda'] = data['otra_moneda'].astype(str)
# data['otra_moneda'] = data['otra_moneda'].astype(str)

In [572]:
# Definir función para estandarizar el pais
def estandarizar_pais(nombre):
    nombre = nombre.strip().lower()
    if re.findall(r"u.s|states|us|america|u. s|u. s.|u.a.|u.s|u.s.|u.sa|ua|u.s.a.|usa|uxz|united|isa|uniter statez|virginia|hartford|san francisco|california|unitef stated", nombre):
        return 'USA'
    elif re.findall(r"united kingdom|uk|kingdom|u.k|great britain|britain|wales|europe|london|scotland|Isle of Man|Cayman Islands",nombre):
        return 'UK'
    elif re.findall(r"canada|canadw|csnada|canda|canad",nombre):
        return 'Canada'
    elif re.findall(r"hong kong|hong konh",nombre):
        return 'Hong Kong'
    elif re.findall(r"netherlands|nederland",nombre):
        return 'Netherlands'
    elif re.findall(r"england|englang",nombre):
        return 'England'
    elif re.findall(r"ireland|loutreland",nombre):
        return 'Ireland'
    elif re.findall(r"spain|catalonia",nombre):
        return 'Spain'
    elif re.findall(r"italy|italia",nombre):
        return 'Italy'
    elif re.findall(r"denmark|danmark",nombre):
        return 'Denmark'
    elif re.findall(r"vietnam|viet nam",nombre):
        return 'Vietnam'
    elif re.findall(r"panama|panamá",nombre):
        return 'Panama'
    elif re.findall(r"brazil|brasil",nombre):
        return 'Brazil'
    elif re.findall(r"mexico|méxico",nombre):
        return 'Mexico'
    elif re.findall(r"india|ibdia",nombre):
        return 'India'
    elif re.findall(r"myanmar|burma",nombre):
        return 'Myanmar'
    elif re.findall(r"czechia|česká republika",nombre):
        return 'Czechia'
    elif re.findall(r"bahamas",nombre):
        return 'Bahamas'
    elif re.findall(r"d'ivoire",nombre):
        return "Côte d'Ivoire"
    elif re.findall(r"china",nombre):
        return 'China'
    elif re.findall(r"africa",nombre):
        return 'Africa'
    elif re.findall(r"luxemburg",nombre):
        return 'Luxembourg'
    elif re.findall(r"other|global|contracts|i.s|remote|international|policy|dbfemf",nombre) or len(nombre)>15 or len(nombre)<3:
        return 'Other'    
    else:
        try:
            return pycountry.countries.lookup(nombre).name
        except LookupError:
            return nombre  

# Cargar el conjunto de datos de ciudades del mundo
cities_df = pd.read_csv('https://raw.githubusercontent.com/datasets/world-cities/refs/heads/main/data/world-cities.csv',delimiter=',')
# Crear una nueva columna con el nombre de la ciudad sin tildes y en minúscula
cities_df['name_adj'] = cities_df['name'].apply(lambda nombre: ''.join(c for c in unicodedata.normalize('NFD', nombre) if unicodedata.category(c) != 'Mn')).str.lower()

# Crear un diccionario para hacer búsquedas rápidas
ciudades_dict = dict(zip(cities_df['name_adj'], cities_df['name']))

# Definir función para estandarizar el nombres de las ciudades
def estandarizar_ciudad(nombre):

    # Quitar las tildes
    nombre = ''.join(c for c in unicodedata.normalize('NFD', nombre) if unicodedata.category(c) != 'Mn')

    if re.findall(r"barxelona",nombre):
        nombre = 'Barcelona'
    elif re.findall(r"nan|remote",nombre):
        nombre = 'otra'
        
    # Eliminar los espacios en blanco y colocar la cadena en minúscula
    nombre = nombre.strip().lower()

    return ciudades_dict.get(nombre, None)

In [573]:
# Limpiar los campos de Pais y Ciudad
data['pais'] = data['pais'].apply(estandarizar_pais)
data['ciudad'] = data['ciudad'].apply(estandarizar_ciudad)

In [574]:
# Definir función para estandarizar otros tipos de monedas
def estandarizar_otra_moneda(moneda):

    # Eliminar los espacios en blanco y colocar la cadena en minúscula
    moneda = moneda.strip().lower()

    if re.findall(r"argentino|argentine|argentinian",moneda):
        moneda = 'ARS'
    elif re.findall(r"indian|rupees",moneda):
        moneda = 'INR'
    elif re.findall(r"mexican|mexicano",moneda):
        moneda = 'MXN'
    elif re.findall(r"norwegian|norwegian",moneda):
        moneda = 'NOK'
    elif re.findall(r"brl|br",moneda):
        moneda = 'BRL'
    elif re.findall(r"american|us dollar",moneda):
        moneda = 'USD'
    elif re.findall(r"philippine|php",moneda):
        moneda = 'PHP'
    elif re.findall(r"australian",moneda):
        moneda = 'AUD'
    elif re.findall(r"ils|israeli ",moneda):
        moneda = 'ILS'
    elif re.findall(r"taiwanese ",moneda):
        moneda = 'TWD'
    elif re.findall(r"pln|polish",moneda):
        moneda = 'PLN'
    elif re.findall(r"czech crowns",moneda):
        moneda = 'CZK'
    elif re.findall(r"rmb|chinese",moneda):
        moneda = 'RMB'
    elif re.findall(r"krw|korean",moneda):
        moneda = 'KRW'
    elif re.findall(r"danish",moneda):
        moneda = 'DKK'
    elif re.findall(r"euro",moneda):
        moneda = 'EUR'
    elif re.findall(r"thai",moneda):
        moneda = 'THB'
    elif re.findall(r"croatian",moneda):
        moneda = 'HRK'
    elif re.findall(r"singapore",moneda):
        moneda = 'SGD'
    elif re.findall(r"colombianos|COP",moneda):
        moneda = 'COP'
    elif re.findall(r"n/a|na",moneda):
        moneda = None
    elif len(moneda) == 3 or len(moneda) == 2:
        moneda = moneda.upper()
    return moneda


In [575]:
# Estandarizar tipo de moneda     
data['otra_moneda'] = data['otra_moneda'].astype(str).apply(estandarizar_otra_moneda)

In [576]:
# Definir tasas de cambio a COP
tasa_cambio = pd.read_excel('data/tasas_cambio.xlsx')

c:\Users\sonia.olaya\AppData\Local\anaconda\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Web Extension extension is not supported and will be removed
  for idx, row in parser.parse():


In [577]:
# Agrupar el tipo de moneda
data['moneda_agg'] = data.apply(lambda x: x['otra_moneda'] if x['moneda'] == 'Other' else x['moneda'], axis=1)

In [578]:
# Definir función que recibe como parámetros el monto y la moneda, y realiza el cambio a COP
def convertir_a_cop(monto, moneda):
    if moneda != 'COP':
        tasa = tasa_cambio.loc[tasa_cambio['currency'] == moneda, 'rate']
        return monto * tasa.iloc[0] if not tasa.empty else monto

    else:
        return monto

In [579]:
# Crear nuevos campos salario anual y compensaciones convirtiendo sueldos y compensaciones a Pesos Colombianos
data['salario_anual_cop'] = data.apply(lambda x: convertir_a_cop(x['salario_anual'],x['moneda_agg']),axis=1).astype(float)
data['compesaciones_cop'] = data.apply(lambda x: convertir_a_cop(x['compensaciones'],x['moneda_agg']),axis=1).astype(float)

In [580]:
# Crear nuevo campo salario anual mas compensaciones
data['salario_total_cop'] = data['salario_anual_cop'] + data['compesaciones_cop']

In [581]:
# data[['edad','industria','titulo_profesional','moneda_agg','pais','ciudad','años_experiencia_total','años_experiencia_industria','nivel_educacion','género','raza','salario_anual_cop','compesaciones_cop','salario_total_cop']].to_csv('data/base_limpia.csv',index=False,sep=',')